In [ ]:
import keras
keras.__version__

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input/dogs-vs-cats"))
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# **Split data**

In [ ]:
import zipfile

with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")

with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")

!ls ./

In [ ]:
original_dataset_dir = './train'
base_dir = '../data'
if not os.path.isdir(base_dir): os.mkdir(base_dir)

In [ ]:
train_dir = os.path.join(base_dir,'train')
if not os.path.isdir(train_dir): os.mkdir(train_dir)
test_dir = os.path.join(base_dir,'test')
if not os.path.isdir(test_dir): os.mkdir(test_dir)
validation_dir = os.path.join(base_dir,'validation')
if not os.path.isdir(validation_dir): os.mkdir(validation_dir)

In [ ]:
train_cats_dir = os.path.join(train_dir,'cats')
if not os.path.isdir(train_cats_dir): os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir,'dogs')
if not os.path.isdir(train_dogs_dir): os.mkdir(train_dogs_dir)
    
validation_cats_dir = os.path.join(validation_dir,'cats')
if not os.path.isdir(validation_cats_dir) : os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir,'dogs')
if not os.path.isdir(validation_dogs_dir) : os.mkdir(validation_dogs_dir)

test_cats_dir = os.path.join(test_dir,'cats')
if not os.path.isdir(test_cats_dir) : os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir,'dogs')
if not os.path.isdir(test_dogs_dir) : os.mkdir(test_dogs_dir)

In [ ]:
def showdir(path, depth):
    if depth == 0:
        print("root:[" + path + "]")
 
    for item in os.listdir(path):
        if '.git' not in item:
            print("|      " * depth + "|--" + item)
 
            newitem = os.path.join(path,item)
            if os.path.isdir(newitem):
                showdir(newitem, depth +1)
showdir(base_dir,0)

In [ ]:
import shutil
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir,fname)
    shutil.copyfile(src,dst)   
fnames = ['cat.{}.jpg'.format(i) for i in range(1000,1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir,fname)
    shutil.copyfile(src,dst)
fnames = ['cat.{}.jpg'.format(i) for i in range(1500,2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir,fname)
    shutil.copyfile(src,dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir,fname)
    shutil.copyfile(src,dst)   
fnames = ['dog.{}.jpg'.format(i) for i in range(1000,1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir,fname)
    shutil.copyfile(src,dst)
fnames = ['dog.{}.jpg'.format(i) for i in range(1500,2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir,fname)
    shutil.copyfile(src,dst)

In [ ]:
import pandas as pd
cats = pd.Series([len(os.listdir(train_cats_dir)),len(os.listdir(validation_cats_dir)),len(os.listdir(test_cats_dir))])
dogs = pd.Series([len(os.listdir(train_dogs_dir)),len(os.listdir(validation_dogs_dir)),len(os.listdir(test_dogs_dir))])
df = pd.DataFrame({'Cats':cats,'Dogs':dogs})
df.index = ['Train','Validation','Test']
df

# **VGG16**

I use a convolutional base layer of a VGG16 network trained on the ImageNet dataset to extract useful features from dog and cat images. Then I'll train the Dogs vs. Cats classifier with this features.

Let's create a VGG16 model:

In [ ]:
import tensorflow as tf
import keras

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

The VGG16 function takes three parameters: `weights`, `include_top`, `input_shape`

Below is the detailed structure of the VGG16 convolutional base layer.

In [ ]:
conv_base.summary()

The size of the final feature map is `(4, 4, 512)`. On top of this feature, I will put a fully connected layer. Two methods are available at this point.

1. Run the convolutional base layer on the new dataset and save the output to disk as a NumPy array. Use this data as input to an independent fully connected classifier.

2. Build a Dense layer on top of the prepared model(In this notebook, `conv_base`) and expand it. It then runs the entire model end-to-end on the input data.

I will try both methods.

# **1. Feature extract without data augmentation (fast, low cost, risk of overfitting)**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

The size of the extracted features is `(samples, 4, 4, 512)`. To put it in the fully connected classifier, expand it to size `(samples, 8192)`:

In [ ]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

Then I define a fully connected classifier and train it using the stored data and labels:

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

A verification accuracy of 90% was reached. But even with a lot of dropout, it overfitted almost immediately as soon as it started training. This is because I did not use data augmentation, which is essential to avoid overfitting on small image datasets.

# **2. Feature extract using data augmentation (slow, high cost, less overfitting)**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(keras.Input(shape=(150,150,3)))
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Extend the `conv_base` model I made in step 1. Models work just like layers, so I can add other models to Sequential model like layers.

The structure of this model is this:

In [ ]:
model.summary()

The convolution-based layer of VGG16 has 14714688 parameters, while the classifier added on top of the convolution-based layer has 2097408 parameters.
    

    
Since I will be using a pre-trained model, the weights should not be updated in the training from now on. So I need to freeze the Convolutional Layer before compiling and training the model.   

A frozen layer is not updated with weights during training. If you don't freeze here, it's useless to use a pre-trained model.

In [ ]:
conv_base.trainable = False

Now I can start training the model using the data augmentation:

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

In [ ]:
img = image.load_img('./train/cat.1665.jpg', target_size=(150,150))
x=image.img_to_array(img)
x=x.reshape((1,) + x.shape)

i=0 
for batch in train_datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot=plt.imshow(image.array_to_img(batch[0]))
    i+=1
    if i%6==0:
        break
plt.show()

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

The verification accuracy is similar to the previous graph. However, the overfitting is slightly reduced.

# **Fine tuning**
Fine-tuning is used to further adjusted to the problem given the reuse model.

**Network fine-tuning steps:**
1. Add a new network on top of the pre-trained base network
2. Freeze the underlying network
3. Train the newly added network
4. Unfreeze some layers in the underlying network
5. Train the unfreeze layer and the newly added layer together

Parts 1 to 3 have already been completed while extracting features. Here, I proceed with steps 4 and 5.

In [ ]:
conv_base.summary()

Let's fine-tune the last three convolutional layers. In other words, all layers up to `block4_pool` are frozen, and `block5_conv1`, `block5_conv2`, and `block5_conv3` layers will be trained.

In [ ]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

Now let's start fine-tuning the network. Use the RMSProp optimizer with a lower learning rate.

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

The graph looks irregular. The graph can be expressed smoothly with the 'exponential moving average'.

In [ ]:
def smooth_curve(points, factor=0.8):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

plt.plot(epochs,
         smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot(epochs,
         smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs,
         smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.plot(epochs,
         smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In terms of the accuracy of the graph, it has improved by about 1% compared to the previous graph. However, the loss curve has gotten worse. The reason that the loss can increase even though the accuracy is improved is that it is the distribution of the loss values that affects the accuracy, not the average.

Finally, evaluate the model on the test data.

In [ ]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

In [ ]:
filenames = os.listdir('/kaggle/working/test1')
test_df = pd.DataFrame({'id': filenames})
test_size = test_df.shape[0]

In [ ]:
test_df.shape

In [ ]:
test_generator_final = test_datagen.flow_from_dataframe(
        test_df,
        "/kaggle/working/test1/",
        x_col="id",
        y_col=None,
        target_size=(200, 200),
        batch_size=20,
        class_mode=None
        )

In [ ]:
prediction = model.predict_generator(test_generator_final, steps=50)

In [ ]:
submission = test_df.copy()
submission['id'] = submission['id'].str.split('.').str[0]
submission.to_csv('submission.csv', index=False)